In [78]:
import pandas as pd
import calendar
import joblib
from sklearn.utils import resample

ortopedia = pd.read_csv("D:\Pós Big Data\Aprendizagem de Máquina\Ortopedia_Coluna.csv", delimiter=';')

In [79]:
# Exibe o número de linhas e o número de colunas
print(ortopedia.shape)

(8017, 17)


In [80]:
#Isolar a base de dados com a classe minoritária
minoritaria = ortopedia[ortopedia['Fusão_de_Vértebras']==1]

majoritaria = ortopedia[ortopedia['Fusão_de_Vértebras']==0]

minoritaria_upsample = resample(minoritaria,
                                replace=True,
                                n_samples=7900,
                                random_state=None
                               )

print(minoritaria_upsample)

     Sexo   CID  Idade  c32020023  c52010465  c52250091  c36010022  c25060082  \
3192    M  M545     65          0          0          2          0          0   
6286    F  M545     71          0          0          2          0          0   
2245    M  Z003     53          0          0          0          0          0   
6255    F  T12      40          2          0          0          0          0   
394     F  M77      60          0          0          2          0          2   
987     F  M545     51          0          0          0          0          0   
2502    M  M545     42          0          0          0          0          0   
4978    F  Z000     56          4          2          2          2          0   
3155    F  M545     77          0          0          2          0          0   
1309    F  S637     44          2          0          2          4          0   
2096    M  M531     41          4          0          2          2          2   
3671    M  M99      63      

In [81]:
#Merge dos Data Frames
#ortopedia = ortopedia.drop(ortopedia[ortopedia['Fusão_de_Vértebras']==1].index)

ortopedia_balanceado = pd.concat([majoritaria,minoritaria_upsample])

print(ortopedia_balanceado['Fusão_de_Vértebras'].value_counts)

<bound method IndexOpsMixin.value_counts of 0       0
1       0
2       0
3       0
4       0
5       0
6       0
7       0
8       0
9       0
10      0
11      0
12      0
13      0
14      0
15      0
16      0
17      0
18      0
19      0
20      0
21      0
22      0
23      0
24      0
25      0
26      0
27      0
28      0
29      0
       ..
582     1
1382    1
1131    1
987     1
6286    1
70      1
6255    1
2096    1
6286    1
3222    1
3828    1
1744    1
4288    1
4978    1
4556    1
1131    1
7291    1
3113    1
4618    1
3192    1
2245    1
4978    1
7291    1
7936    1
3828    1
1200    1
3671    1
7291    1
2096    1
2245    1
Name: Fusão_de_Vértebras, Length: 15883, dtype: int64>


In [82]:
# Divide os dados em dois conjuntos: Atributos e Classes
attributes = ortopedia_balanceado.drop('Fusão_de_Vértebras', axis=1)
classes = ortopedia_balanceado['Fusão_de_Vértebras']

In [83]:
#Cria dicionário e mapa para meses
#d = {'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4, 'may': 5, 'jun': 6, 'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11,
#     'dec': 12}
#attributes['month'] = attributes['month'].map(d).astype(int)

#Cria atributos "dummies" para as colunas que não são numericas no conjunto de dados
new_attributes = pd.get_dummies(attributes)
print(new_attributes)

      Idade  c32020023  c52010465  c52250091  c36010022  c25060082  c32020015  \
0        40          0          0          0          2          0          0   
1        11          0          0          0          0          0          2   
2        68          2          0          0          0          0          0   
3        49          0          0          0          0          0          0   
4        46          0          0          0          0          0          0   
5        45          0          0          0          0          0          2   
6        44          0          0          0          0          0          0   
7        15          2          0          0          0          0          0   
8        31          2          0          0          0          0          4   
9        13          0          0          0          0          0          0   
10       34          0          0          0          0          0          0   
11       19          0      

In [84]:
# Dividir os dados aleatoriamente em conjunto para aprendizado e conjunto para testes
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(new_attributes, classes, test_size=0.20) 
#20% do tamanho do arquivo será usado para testes
# X_train: segmento dos atributos para treinamento do modelo
# X_test : segmento dos atributos para avaliação do modelo
# y_train: segmento das classes para treinamento do modelo
# y_testn: segmento das classes para avaliação do modelo

In [85]:
#Treinar o modelo
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X_train, y_train)

C:\Users\rudso\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [86]:
#Aplicar o modelo gerado sobre os dados separados para testes
y_pred = classifier.predict(X_test)
y_pred_proba = classifier.predict_proba(X_test)

print(y_pred)
print(y_pred_proba)

[1 1 1 ... 0 1 1]
[[6.06742412e-09 9.99999994e-01]
 [2.79838068e-03 9.97201619e-01]
 [6.06742412e-09 9.99999994e-01]
 ...
 [9.75659098e-01 2.43409023e-02]
 [5.36623033e-03 9.94633770e-01]
 [3.33341057e-06 9.99996667e-01]]


In [87]:
#Avaliar o modelo: Acurácia e matriz de contingência
from sklearn.metrics import classification_report, confusion_matrix
print("Resultado da Avaliação do Modelo")
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Resultado da Avaliação do Modelo
[[1556   45]
 [   0 1576]]
              precision    recall  f1-score   support

           0       1.00      0.97      0.99      1601
           1       0.97      1.00      0.99      1576

   micro avg       0.99      0.99      0.99      3177
   macro avg       0.99      0.99      0.99      3177
weighted avg       0.99      0.99      0.99      3177



In [32]:
#Classificar uma nova instância
print("Classificar [5, 177, 72, 11, 500, 37.6, 0.256, 47]")
nova_instancia=[[5, 177, 72, 11, 500, 37.6, 0.256, 47]]
print(classifier.predict(nova_instancia))

Classificar [5, 177, 72, 11, 500, 37.6, 0.256, 47]
['tested_positive']


In [88]:
#Salvar o modelo para uso posterior
joblib.dump(classifier, 'regr_logist_ortopedia.joblib')

['regr_logist_ortopedia.joblib']

In [48]:
classifier = joblib.load('regr_logist_ortopedia.joblib')
nova_instancia=[[5, 177, 72, 11, 500, 37.6, 0.256, 47]]
print("Com o modelo salvo: ")
print("Classificar [5, 177, 72, 11, 500, 37.6, 0.256, 47]")
print("Predict")
print(classifier.predict(nova_instancia))
print("Proba")
print(classifier.predict_proba(nova_instancia))

Com o modelo salvo: 
Classificar [5, 177, 72, 11, 500, 37.6, 0.256, 47]
Predict
['tested_positive']
Proba
[[0. 1.]]


In [ ]:
### Análise

Qual é a acurácia do modelo?
A acurácia do modelo é de 1,00 para os registros do tipo "Não precisa de fusão de vértebra", 
e 0,97 para os registros do tipo "Precisa de fusão de vértebra".

Qual é a função gerada pelo modelo?
Não recebemos instruções para gerar esta informação.